(Written by Amir Akhavan)

In [11]:
import requests
import pandas as pd
import json
from pathlib import Path

In [162]:
from bs4 import BeautifulSoup
from tqdm import tqdm

In [2]:
PREPROCESSED_DATA_PATH = Path('../.../data/preprocessed/property_assessor')

In [3]:
county_fips_code = '45039'

In [4]:
initial  = "https://beacon.schneidercorp.com/Application.aspx?AppID=796&LayerID=16296"
x_initial = requests.get(initial)
x_initial = x_initial.content.decode("utf-8")

In [5]:
qps_location = x_initial.find('QPS')
qps_closing_str = x_initial[qps_location+6:]
end = qps_closing_str.find('"')
QPS = qps_closing_str[:end]

In [13]:
QPS

'F9tZvOIY_XZOdUskmaGw54VoDEVPL0bY8E9muKXf1q0Y3RtiKO-fXkr2WvfxscDkbVAKnYudfhVepVVn-oXYgP-dqgsX8get-bxXhSkh4h4NzFwuFmgkFApZxPKkUjTUOnZS8m05u_47GWkMiLArf-BELR2u4O_IsifbxqlkvNtpyWSI9NEdFPBGUG59ko3ZnbGXYYaGVTaUEWlA9LDwawoz8bXObWcDd2mFX2k6SDnSaEkr5mgwe-AqziPV5IUfvXBMc1qOAcgs1In8v_8iYw2'

In [137]:
url = f'https://qpublic.schneidercorp.com/api/beaconCore/BufferAndExport?QPS={QPS}'
myobj = {
    'layerId': '11834', # '16297', #  original: "11834",
    'keys': ["001-00-00-003-000", "150-00-00-004-000"],
    'distance': 10000000,
    'exportType': "xlsx",
    # 'keys': ["001-00-00-003-000", "028-00-00-004-000"],
    'showParcelId': 'true',
    'skipLabelCount': "0",
    'usePropertyAddress': 'true',
    # my additions:
    #'Visible': 'true',
    #'LayerName': 'Parcels',
    #'Query': 'true',
    #"HasMap":'true',
}
x = requests.post(url, data = myobj)
url_of_csv="https://beacon.schneidercorp.com/"

In [138]:
# x.content

In [139]:
str_vals = json.loads(x.content)
download_url = url_of_csv+str_vals['d']['DownloadUrl']
df = pd.read_excel(download_url)
df = df.drop_duplicates()

In [140]:
# str_vals['d']

In [141]:
geometry = str_vals['d']['BufferGeometry'].split('MULTIPOLYGON')[1].replace('(', '[').replace(')', ']')

In [142]:
df

,ParcelId,OwnerName,Address,City,State,Zip
0,216-00-00-005-000,CURRENT RESIDENT,2100FT W OF INT S-20-323 & S-20-324,Fairfield,SC,NaN
1,215-00-00-074-000,CURRENT RESIDENT,293 EMERALD DR TRACT,Fairfield,SC,NaN
2,215-00-00-102-000,CURRENT RESIDENT,285 EMERALD DR,Fairfield,SC,NaN
3,026-04-02-005-000,CURRENT RESIDENT,SW OF GIBSON END RD RD,Fairfield,SC,NaN
4,026-04-02-007-000,CURRENT RESIDENT,557 GIBSON END ROAD,Fairfield,SC,NaN
...,...,...,...,...,...,...
17480,141-00-00-031-000,CURRENT RESIDENT,S 2000FT INT S-20-115 & HWY 34 E H,Fairfield,SC,NaN
17481,077-01-00-002-000,CURRENT RESIDENT,1704 GREAT NORTH RD RD,Fairfield,SC,NaN
17482,048-00-00-002-000,CURRENT RESIDENT,"HWY 215 N, CLOWNEY & JOHN BRICE RDS",Fairfield,SC,NaN
17483,047-00-00-047-000,CURRENT RESIDENT,JOHN BRICE RD,Fairfield,SC,NaN


In [143]:
#df = df.set_index('ParcelId')
#df

In [144]:
parcel_ids = df['ParcelId']
parcel_ids

0        216-00-00-005-000
1        215-00-00-074-000
2        215-00-00-102-000
3        026-04-02-005-000
4        026-04-02-007-000
               ...        
17480    141-00-00-031-000
17481    077-01-00-002-000
17482    048-00-00-002-000
17483    047-00-00-047-000
17490    114-01-01-021-000
Name: ParcelId, Length: 17244, dtype: object

In [145]:
def lookup_parcel_id(parcel_id):
    property_lookup_url = 'https://beacon.schneidercorp.com/Application.aspx?AppID=796&LayerID=11834&PageTypeID=4&PageID=5738&Q=1649139588&KeyValue='
    url = property_lookup_url + parcel_id
    html = requests.get(url).content.decode("utf-8")
    return html

In [155]:
def market_value(parcel_html):
    try:
        #'Total Market Value' in html
        idx = parcel_html.index('Total Market Value')
        substring = parcel_html[idx:]
        idx = substring.index('$')
        substring = substring[idx:]
        idx = substring.index('</td>')
        value_str = substring[1:idx]
        return float(value_str.replace(',', ''))
    except:
        return float('NaN')

In [ ]:
market_values = []
for i, parcel_id in tqdm(enumerate(parcel_ids)):
    parcel_html = lookup_parcel_id(parcel_id)
    value = market_value(parcel_html)
    market_values.append(value)
    # print(df.iloc[i]) # ['MarketValue'] = value
df['MarketValue'] = market_values
df

238it [00:30,  9.11it/s]

In [8]:
path = PREPROCESSED_DATA_PATH / 'fips' / county_fips_code
!mkdir -p {path}
df.to_csv(path / 'properties.csv')

In [9]:
!wc -l {path}/properties.csv

17245 ../../data/preprocessed/property_assessor/fips/45039/properties.csv


In [10]:
df

,ParcelId,OwnerName,Address,City,State,Zip
0,216-00-00-005-000,CURRENT RESIDENT,2100FT W OF INT S-20-323 & S-20-324,Fairfield,SC,NaN
1,215-00-00-074-000,CURRENT RESIDENT,293 EMERALD DR TRACT,Fairfield,SC,NaN
2,215-00-00-102-000,CURRENT RESIDENT,285 EMERALD DR,Fairfield,SC,NaN
3,026-04-02-005-000,CURRENT RESIDENT,SW OF GIBSON END RD RD,Fairfield,SC,NaN
4,026-04-02-007-000,CURRENT RESIDENT,557 GIBSON END ROAD,Fairfield,SC,NaN
...,...,...,...,...,...,...
17480,141-00-00-031-000,CURRENT RESIDENT,S 2000FT INT S-20-115 & HWY 34 E H,Fairfield,SC,NaN
17481,077-01-00-002-000,CURRENT RESIDENT,1704 GREAT NORTH RD RD,Fairfield,SC,NaN
17482,048-00-00-002-000,CURRENT RESIDENT,"HWY 215 N, CLOWNEY & JOHN BRICE RDS",Fairfield,SC,NaN
17483,047-00-00-047-000,CURRENT RESIDENT,JOHN BRICE RD,Fairfield,SC,NaN
